# 1 Introduction
Memory Networks (MN) is introduced by Jason Weston et al. in 2014 to solve question-answering tasks [1]. MN is a neural network architecture which is accepting several pieces of evidence (sentences) and questions, answering them after finishing updating memories. They applied fixed hops to read evidences and updating memories and hard attention (chosing one explicit sentence) to update memories and eventually generate answers. In these project, I propose a new mechanism with hard/soft attentions and stop criteria based on reinforcement learning approaches (DQN). Evolutionary function approximation and several learning methods will be tested under the structure of MN. Facebook's bAbI dataset will be used to evaluate the new mechanism.

The structure of this report is as follows.

Section 2 reviews related work on Memory Networks and Deep Q-learning. Section 3 introduces my new Memory Networks mechanism combining DQN with End2End Memory Networks. Section 4 evaluates the new model by using Facebook's bAbi dataset. Section 5 discusses the future work.

# 2 Related Work
Question answering (QA) is a complex natural language processing task which requires a deep understanding of a text and the ability to reason over relevant facts.For example, given following factes,
> _John picked up the apple._

> _John went to the office._

> _John went to the kitchen._

> _John dropped the apple._

> _Mary travelled to the office._

The question is _Where was the apple before the kitchen?_ . In order to answer this question, firstly the fact _John picked up the apple._ is picked up, and then _John went to the kitchen._ & _John went to the office._ are selected to find the correct location of the apple. From this case, the basic reasonning process can be briefly described as follows.
1. Read a question!
2. Read facts.
3. Have new knowledge.
4. Back to (2) or answer the question
-------------------------------------------

## 2.1 Memory Networks
Likewise, the End2End Memory Networks has the same procedure and can be described by 4 components[2]:

1. **Read a question**, the input feature map $ I $ is used to converts the question into a input feature representation $ u $.
2. **Read facts**, the input memory representation takes the inner product of $ u $ and each memory $ m_i $ followed by a softmax:
$$
p_i = Softmax(u^T m_i) 
$$
3. **Have and new knowledge**, the output memory representation sums over the transformed inputs $ c_i $ weighted by the probability vector $ p_i $,
$$
o = \sum_i {p_i c_i}
$$
4. **Answer the question**, the final prediction is from the the sum of the output vector $ o $ and the input embedding $ u $ through a weight matrix $ W $ and a softmax:
$$
\hat{a} = Softmax(W(o + u)) 
$$
![alt text](memn2n.png "Strucutre of MemN2N")
However, there are two issues need to be considered. One of them is when stop updating $ u $ to predict answers (how many hops are needed).Intuitively, to use reinforcement learning to decide the next action (continue reading&updatig or stop to predict answers) enable the model to find the best choices. The other issue is how to balance exploitation and exploration if there are a large set of facts (can not explicitly have $ A $) at the time to read information from facts. Until now, this project has only finished testing the first idea, and the second one will be done in the near future.

## 2.2 DQN
Deep Q-learning (DQN) is the q-learning with a deep neural networks as its q-value estimation function. The loss function is shown as follows.
$$
L = [r + \gamma \max_{a'}Q(S',a') - Q(s,a)]^2
$$
Gieven a transition $(s,a,r,s')$, DQN use the neural networks to estimate $Q(s,a)$.
$$
Q(s,a) = f(s, \theta)
$$

# 3 QMemN2N
In the section, I propose a new mechanism (QMemN2N) which applies DQN to control the reading action of MemN2N. The states of QMemN2N is $ u_1, u_2, ..., u_n $; the actions are 0 (continue) and 1 (stop); the rewards are 1.0 when answer is correct, -1.0 when answer is wrong and 0 when the process continues to next hop. Here is the basic algorithm,
~~~
Train MemN2N, different hops share the same weighted vector $ A $ for reading and $ C $ for writing;
Initialize QMemN2N by copying input feature map $ I $, weight matrix $ W $,  $A$ and $C$
Train DQN, 20 hops maximum
~~~
![alt text](qmem.png "Procedure of QMemN2N")

## 3.1 Implementation
The basic implementation of MemN2N is from [domluna's work](https://github.com/domluna/memn2n). I modified its model structure from different $C$ for different hops to unique $C$ for all hops. And then I build the QMemN2N shown in [Appendix](#Appendix)

# 4 Experiments
The dataset bAbi (with 20 different tasks) is used to test QMemN2N. Firstly, I run MemN2N on different numbers of hops from 1 to 20 repectively. And then the QMemN2N model is built on the MemN2N with 2 hops. 
In the task 9 (Simple Negation), QMemN2N performs better than all MemN2N models, and the result is shown here.
![alt text](task9.png "Results on Task 9")
In the task 15 (Basic Deduction), QMemN2N performs much better than the MemN2N models with 2 hops, and the result is shown here.
![alt text](task15.png "Results on Task 15")
The next table shows the results on all tasks. The results of MemN2N is based on 2 hops, while the results of QMemN2N is based on dynamic hops decided by DQN.
![alt text](result.png "Results on All Tasks")
From this table, not all QMemN2N are better than MemN2N (QMemN2N works better on 11 of them), but still this is an encouraging results.

# 5 Future Work
In this project, I tested my first idea to dynamically stop to predict the answers. However, considering the data in bAbi is well separated into 20 categories, more experiments with mixed questions are needed. Also, right now I only trained the q-value estimator on current $u$, so more powerful estimators on multiple previous $u_{i-k}, ..., u_k$ and the difference between $u_k$ and $u_0$ are worth trying. Beside, to balance exploitation and exploration is more challenging but is with more potential.

# Appendix
The complete code are in file _QMemN2N.py_. Here, I only show the core code.
~~~python
model = MemN2N(batch_size, vocab_size, sentence_size, memory_size, FLAGS.embedding_size, session=sess,
               hops=FLAGS.hops, max_grad_norm=FLAGS.max_grad_norm)
for t in range(1, FLAGS.epochs+1):
    # Stepped learning rate
    if t - 1 <= FLAGS.anneal_stop_epoch:
        anneal = 2.0 ** ((t - 1) // FLAGS.anneal_rate)
    else:
        anneal = 2.0 ** (FLAGS.anneal_stop_epoch // FLAGS.anneal_rate)
    lr = FLAGS.learning_rate / anneal
    #Train MemN2N
    np.random.shuffle(batches)
    total_cost = 0.0
    for start, end in batches:
        s = trainS[start:end]
        q = trainQ[start:end]
        a = trainA[start:end]
        cost_t = model.batch_fit(s, q, a, lr)
        total_cost += cost_t
    if t % FLAGS.evaluation_interval == 0:
        train_preds = []
        for start in range(0, n_train, batch_size):
            end = start + batch_size
            s = trainS[start:end]
            q = trainQ[start:end]
            pred = model.predict(s, q)
            train_preds += list(pred)

        val_preds = model.predict(valS, valQ)
        train_acc = metrics.accuracy_score(np.array(train_preds), train_labels)
        val_acc = metrics.accuracy_score(val_preds, val_labels)

        print('-----------------------')
        print('Epoch', t)
        print('Total Cost:', total_cost)
        print('Training Accuracy:', train_acc)
        print('Validation Accuracy:', val_acc)
        print('-----------------------')
#prediction on MemN2N
test_preds = model.predict(testS, testQ)
test_acc = metrics.accuracy_score(test_preds, test_labels)
print("Testing Accuracy:", test_acc)
#initialize QMemN2N
q_model = QMemN2N(model, 0.8)
#train QMemN2N
for t in range(1, FLAGS.epochs+1):
    for i in range(n_train):
        u0 = q_model.getFirstState(trainQ[i])
        #get u1
        r,u1 = q_model.takeAction(0, trainS[i], trainA[i], u0)
        G = r
        for j in range(20):
            a = q_model.qvalue().argmax()
            if np.random.random() < epsilon:
                a = np.random.choice(range(2))
            r,u2 = q_model.takeAction(a, trainS[i], trainA[i], u1)
            G += r
            q_model.updateQfunction(r, u1, a)
            u1 = u2 #new states
            if a == 1:
                break
~~~

# Reference

[1] Weston, Jason, Sumit Chopra, and Antoine Bordes. "Memory networks." arXiv preprint arXiv:1410.3916 (2014).

[2] Sukhbaatar, Sainbayar, Jason Weston, and Rob Fergus. "End-to-end memory networks." Advances in neural information processing systems. 2015.

[3] Mnih, Volodymyr, et al. "Playing atari with deep reinforcement learning." arXiv preprint arXiv:1312.5602 (2013).